In [2]:
import os
import pandas as pd
import json

In [ ]:
datasets_directory="classification_datasets"
output_directory="../LLM/eeg_datasets"

os.makedirs(output_directory,exist_ok=True)


In [4]:
def dataset_formatting(dataframe,columns_to_drop):
    dataset=[]
    for i, row in dataframe.drop(columns=columns_to_drop).iterrows():
        sample={}
        prompt="Quantitative EEG: "
        for key,value in row.items():
            if(key != "Class"):
                prompt+=f"{key} = {value}, "
        completion="Healthy" if row["Class"] == 0 else "ALS"
        sample["prompt"]=prompt[:-2]
        sample["completion"]=completion
        
        dataset.append(sample)
    
    return dataset

In [5]:
def save_jsonl(data, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for item in data:
            f.write(json.dumps(item) + "\n")

In [8]:
columns_to_drop=["PatientId"]
for ds in os.listdir(datasets_directory):
    ds_path=os.path.join(datasets_directory,ds)
    for trial in os.listdir(ds_path):
        ds_input_path=os.path.join(datasets_directory,ds,trial)
        ds_output_path=os.path.join(output_directory,ds,trial)

        train_df=pd.read_csv(os.path.join(ds_input_path,"train.csv"))
        val_df=pd.read_csv(os.path.join(ds_input_path,"val.csv"))
        test_df=pd.read_csv(os.path.join(ds_input_path,"test.csv"))


        train_df_formatted=dataset_formatting(train_df,columns_to_drop)
        val_df_formatted=dataset_formatting(val_df,columns_to_drop)
        test_df_formatted=dataset_formatting(test_df,columns_to_drop)
        
        os.makedirs(output_directory,exist_ok=True)
        os.makedirs(os.path.join(output_directory,ds),exist_ok=True)
        os.makedirs(ds_output_path,exist_ok=True)

        save_jsonl(train_df_formatted,os.path.join(ds_output_path,"train.jsonl"))
        save_jsonl(val_df_formatted,os.path.join(ds_output_path,"val.jsonl"))
        save_jsonl(test_df_formatted,os.path.join(ds_output_path,"test.jsonl"))